## 12/11
- 肥満、高脂血症、高血糖、脂質異常（MetSにおいて）を判定
- メタボの判定


In [2]:
# import master_code as mc
import numpy as np
import pandas as pd
main = pd.read_csv('/Volumes/USBDISK/AHHDC/git/html/dummy2.csv', encoding = 'cp932', index_col = 0)
#　変数なし

df = main.copy()

def mets_judge(df):   # 変数最小
    #腹囲判定
    df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True       
                            .mask((df['性別'] == '女') & (df['腹囲'] >= 90), True) # 女性    　
                            .mask((df['性別'] == '男') & (df['腹囲'] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df['性別'] == '女') & (df['腹囲'] < 90), False)  # 女性
                            .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
       )

    #血圧判定
    df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), True)      # どちらかがTrueの場合True
                              .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), False)            # どちらもFalseの場合False
                              .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                  & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #血糖判定
    df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), True)          # どちらかがTrueの場合True
                              .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), False)                # どちらもFalseの場合False
                              .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #脂質判定
    df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), True)           # どちらかがTrueの場合True
                               .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), False)               # どちらもFalseの場合False
                               .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    df['MetS'] = (df['腹囲'] 
                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある                         # 4-3
                       , 'N/A')                                                                         # 判定なし         

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or                       # 3-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲異常　&
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &                                    # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
                  
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))             # 1-1, 1-2
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
       )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

In [ ]:
df = main.copy()
metS_facter = [
    ['性別', '腹囲'],               # 腹囲
    ['収縮機血圧', '拡張期血圧'],     # 血圧
    ['空腹時血糖', 'HbA1c'],        # 血糖
    ['中性脂肪', 'HDL']             # 脂質
]

def mets_judge(df, metS_facter = [['性別', '腹囲'], ['収縮期血圧', '拡張期血圧'], ['空腹時血糖', 'HbA1c'], ['中性脂肪', 'HDL']]):   #全対応
    #腹囲判定
    df['腹囲_'] = (df[metS_facter[0][1]].mask((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True
                            .mask((df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] >= 90), True) # 女性    　
                            .mask((df[metS_facter[0][0]] == 1) & (df[metS_facter[0][1]] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df[metS_facter[0][0]] == 0) & (df[metS_facter[0][1]] < 90), False)  # 女性
                            .mask((df[metS_facter[0][0]].isnull()) | (df[metS_facter[0][1]].isnull()), np.nan)    #性別 or 腹囲がない場合nan
    )

    #血圧判定
    df['血圧_'] = (df[metS_facter[1][0]].mask((df[metS_facter[1][0]] >= 130) | (df[metS_facter[1][1]] >= 85), True)      # どちらかがTrueの場合True
                            .mask((df[metS_facter[1][0]] < 130) & (df[metS_facter[1][1]] < 85), False)            # どちらもFalseの場合False
                            .mask((~(df[metS_facter[1][0]] >= 130) & ~(df[metS_facter[1][1]] >= 85))                    # どちらもFalseかnanであって
                                  & ((df[metS_facter[1][0]].isnull()) | (df[metS_facter[1][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    #血糖判定
    df['血糖_'] = (df[metS_facter[2][0]].mask((df[metS_facter[2][0]] >= 110) | (df[metS_facter[2][1]] >= 6.0), True)          # どちらかがTrueの場合True
                            .mask((df[metS_facter[2][0]] < 110) & (df[metS_facter[2][1]] < 6.0), False)                # どちらもFalseの場合False
                            .mask((~(df[metS_facter[2][0]] >= 110) & ~(df[metS_facter[2][1]] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df[metS_facter[2][0]].isnull()) | (df[metS_facter[2][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    #脂質判定
    df['脂質_'] = (df[metS_facter[3][0]].mask((df[metS_facter[3][0]] >= 150) | (df[metS_facter[3][1]] <= 40), True)           # どちらかがTrueの場合True
                            .mask((df[metS_facter[3][0]] < 150) & (df[metS_facter[3][1]] > 40), False)               # どちらもFalseの場合False
                            .mask((~(df[metS_facter[3][0]] >= 150) & ~(df[metS_facter[3][1]] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df[metS_facter[3][0]].isnull()) | (df[metS_facter[3][1]].isnull())), np.nan)      # どちらもnanの場合nan
    )

    df['MetS'] = (df['腹囲']
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))           
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or
                     ((df['腹囲_'] == True) & (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &             # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある
                       , 'N/A')                                                                         # 判定なし                 
    )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

1   1,  1,  1,      MetS 　         1-1

1   1,  1,  nan     MetS            1-2

1   1,  0,  nan     can't 　        4-3

1   1,  nan,nan     can't 　        4-3

1   nan,nan,nan     can't 　        4-2

1   1,  0,  0       s/o 　          2-1
 　
0   1,  1,  1       not 　          3-1

0   nan,nan,nan     not 　          3-2

nan ~,  ~,  ~       can't           4-1


In [3]:
#　変数なし

df = main.copy()

def mets_judge(df):   # 変数最小
    #腹囲判定
    df['腹囲_'] = (df['腹囲'].mask((df['性別'] == '男') & (df['腹囲'] >= 85), True) # 男性       #どちらもデータがあってTrueの場合True       
                            .mask((df['性別'] == '女') & (df['腹囲'] >= 90), True) # 女性    　
                            .mask((df['性別'] == '男') & (df['腹囲'] < 85), False)  # 男性      #どちらもデータがあってFalseの場合False
                            .mask((df['性別'] == '女') & (df['腹囲'] < 90), False)  # 女性
                            .mask((df['性別'].isnull()) | (df['腹囲'].isnull()), np.nan)    #性別 or 腹囲がない場合nan
       )

    #血圧判定
    df['血圧_'] = (df['収縮期血圧'] .mask((df['収縮期血圧'] >= 130) | (df['拡張期血圧'] >= 85), True)      # どちらかがTrueの場合True
                              .mask((df['収縮期血圧'] < 130) & (df['拡張期血圧'] < 85), False)            # どちらもFalseの場合False
                              .mask((~(df['収縮期血圧'] >= 130) & ~(df['拡張期血圧'] >= 85))                    # どちらもFalseかnanであって
                                  & ((df['収縮期血圧'].isnull()) | (df['拡張期血圧'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #血糖判定
    df['血糖_'] = (df['空腹時血糖'].mask((df['空腹時血糖'] >= 110) | (df['HbA1c'] >= 6.0), True)          # どちらかがTrueの場合True
                              .mask((df['空腹時血糖'] < 110) & (df['HbA1c'] < 6.0), False)                # どちらもFalseの場合False
                              .mask((~(df['空腹時血糖'] >= 110) & ~(df['HbA1c'] >= 6.0))                   # どちらもFalseかnanであって
                                  & ((df['空腹時血糖'].isnull()) | (df['HbA1c'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    #脂質判定
    df['脂質_'] = (df['中性脂肪'].mask((df['中性脂肪'] >= 150) | (df['HDL'] <= 40), True)           # どちらかがTrueの場合True
                               .mask((df['中性脂肪'] < 150) & (df['HDL'] > 40), False)               # どちらもFalseの場合False
                               .mask((~(df['中性脂肪'] >= 150) & ~(df['HDL'] <= 40))                 # どちらもFalseかnanであって     
                                  & ((df['中性脂肪'].isnull()) | (df['HDL'].isnull())), np.nan)      # どちらもnanの場合nan
       )

    df['MetS'] = (df['腹囲'] 
                .mask((df['腹囲_'].isnull()) |                                                           # 腹囲がない or                      # 4-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲がある &
                     ((df['血圧_'].isnull()) & (df['血糖_'].isnull()) & (df['脂質_'].isnull()) |          # 3項目がない(nanがある) or           # 4-2 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                     ((df['血圧_'].isnull()) | (df['血糖_'].isnull()) | (df['脂質_'].isnull()))))         # nanがある                         # 4-3
                       , 'N/A')                                                                         # 判定なし         

                .mask((df['腹囲_'] == False) |                                                           # 腹囲正常　or                       # 3-1
                     ((df['腹囲_'] == True) &                                                            # 腹囲異常　&
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 0) &                                    # 3項目が正常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull()))        # 3項目がある(nanがない)              # 3-2
                       , 'not_MetS')                                                                    # 腹囲がない & 異常なし

                .mask((df['腹囲_'] == True) &                                                            # 腹囲異常　& 
                      (df['血圧_'] + df['血糖_'] + df['脂質_'] == 1) &                                    # 1つの項目が異常 &
                      (df['血圧_'].notnull()) & (df['血糖_'].notnull()) & (df['脂質_'].notnull())         # 3項目がある(nanがない)              # 2-1
                       , 's/o_MetS')                                                                    # 腹囲がある & 1つの項目が異常
                  
                .mask((df['腹囲_'] == True) & ((df['血圧_'] + df['血糖_'] + df['脂質_'] >= 2))             # 1-1, 1-2
                       , 'MetS')                                                                        # 腹囲がある & 2つ以上の項目が異常                                                                     
       )
    
    df_metS = df[df['MetS'] == 'MetS']
    df_so_metS = df[df['MetS'] == 's/o_MetS']
    df_non_metS = df[df['MetS'] == 'not_MetS']
    df_na_metS = df[df['MetS'] == 'N/A']
    return df_metS, df_so_metS, df_non_metS, df_na_metS

In [ ]:
mets_judge(df)
df

In [ ]:
# 実行
df_metS, df_so_metS, df_non_metS, df_na_metS = mets_judge(df)
print(df_metS)
print(df_non_metS)

In [ ]:
df.isna().sum()

In [ ]:
# facter_binarization
def facter_binarization(df, facter_th = ['HbA1c', '>= 5.6']):        #カテゴリー変数を２値化、trueが１、true_valueはリスト
    df[f'{facter_th[0]}_'] = list(eval(f"(df['{facter_th[0]}']\
                                       .mask(df['{facter_th[0]}'] {facter_th[1]}, 1)\
                                       .mask(~(df['{facter_th[0]}'] {facter_th[1]}), 0)\
                                       .mask(df['{facter_th[0]}'].isnull(), np.nan))"
                                       ))

## 12/18
- 年度で数字を振り分ける
- ハッシュ値で横並びに

In [3]:
df = main.copy()
mets_judge(df)
implementation_years = np.sort(df['検診実施年度'].unique())                                         # 検診実施年度のリスト
for i, implementation_year in enumerate(implementation_years):
    df_1 = (df[df['検診実施年度'] == implementation_year]                                           # 検診実施年度でフィルター
            .drop(columns = '検診実施年度')                                                         # 検診実施年度を削除
            .rename(columns = lambda x: x if x in ['ID',] else x + f'_{implementation_year}'))    # ID以外の列名に検診実施年度を付与
    if i == 0:
        merge_df = df_1                                                                           # 初回はそのまま代入
    else:
        merge_df = pd.merge(merge_df, df_1, on=['ID', ], how='outer')                             # 2回目以降はIDをキーにして結合

# マージまで
# 何をするか


print(len(merge_df['ID'].unique()))
merge_df

In [13]:
test = merge_df.copy()
column_names = test.columns

In [7]:
column_names = test.columns
gender = [gender for gender in column_names if gender.startswith('性別_')]
gender

In [21]:
delta_metS = [metS for metS in column_names if metS.startswith('MetS_')]
test1 = test[delta_metS]

In [22]:
test1

,MetS_2016,MetS_2017,MetS_2018
0,not_MetS,NaN,not_MetS
1,not_MetS,NaN,NaN
2,not_MetS,NaN,not_MetS
3,not_MetS,NaN,not_MetS
4,not_MetS,NaN,NaN
...,...,...,...
18572,NaN,NaN,s/o_MetS
18573,NaN,NaN,not_MetS
18574,NaN,NaN,not_MetS
18575,NaN,NaN,MetS
